# Apple Identification Model

See [README.md](README.md) for more information on this project.

## Table of Contents
- [Dataset](##dataset)
    - [Filtering Data](###filtering-data)
    - [Data Preprocessing](###data-preprocessing)
- [Model](##model)
    - [Model Architecture](###model-architecture)
    - [Training](###training)
    - [Evaluation](###evaluation)
- [Results](##results)
- [Conclusion](##conclusion)


In [1]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

## Dataset
The dataset used is the Fruits360 dataset from Kaggle, which contains images of various fruits. I will be filtering the dataset to only include apples and their different varieties.

### Filtering Data
The dataset contains a large number of fruit images, but we will only be using the apple images.

In [2]:
kaggle_api = KaggleApi()
kaggle_api.authenticate()

# Download the fruits dataset from Kaggle
kaggle_api.dataset_download_files('moltean/fruits', path='fruits_dataset', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
